In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import csv
import pandas
import pickle
import numpy as np

In [2]:
TRAINING_SIZE = 1000
RO_COUNT = 4
SENSOR_COUNT = RO_COUNT + 1
BLOCK_SIZE = TRAINING_SIZE*SENSOR_COUNT
# const number of instructions in the training set
INSTRUCTIION_COUNT = 54
# This variable gets updated later in the code when we split the multi-cycle instructions
Num_Of_Instructions = INSTRUCTIION_COUNT
# Change this if you use a different clock rate
HERTZ = "100"
# set the following to true if you want to use the reduced instruction groupings
# at most one should be set to True
USEGROUP1 = False
USEGROUP2 = False

In [3]:
# read the data from the csv

csvdata = open("data_" + HERTZ + "_1000.csv")
csvreader = csv.reader(csvdata)
NumOfRows = 0
master_data = []

for row in csvreader:
    if ''.join(row).strip():
        master_data.append(row)
        NumOfRows = NumOfRows + 1

# if (NumOfRows%SENSOR_COUNT) != 0:
#     print("Error Invalid Number of Rows")
        
        
# print(len(master_data[0]))
# print(len(master_data[1]))
# print(len(master_data[2]))
# print(len(master_data[3]))
# print(len(master_data[4]))

print(NumOfRows)
#print(master_data)
csvdata.close()
print("Done")

55
55
55
55
55
270000
Done


In [4]:
# Change the labels
# I regrouped some of the instructions, so this loop reassigns those instructions
# to the correct class/type
# inst, class, type, iteration, brald, add, nop, ori, ori, pre, inst, post, ....

for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
    for j in range(0, TRAINING_SIZE): # TrainingSize
        ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
        ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
        ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
        ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
        ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]
        
        if ro_0_line[0].strip() == "sext8":
            ro_0_line[1] = "Logical"
            ro_1_line[1] = "Logical"
            ro_2_line[1] = "Logical"
            ro_3_line[1] = "Logical"
            ro_sync_line[1] = "Logical"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
        if ro_0_line[0].strip() == "srl":
            ro_0_line[1] = "Logical"
            ro_1_line[1] = "Logical"
            ro_2_line[1] = "Logical"
            ro_3_line[1] = "Logical"
            ro_sync_line[1] = "Logical"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
        if ro_0_line[0].strip() == "sra":
            ro_0_line[1] = "Logical"
            ro_1_line[1] = "Logical"
            ro_2_line[1] = "Logical"
            ro_3_line[1] = "Logical"
            ro_sync_line[1] = "Logical"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
        if ro_0_line[0].strip() == "bsll":
            ro_0_line[1] = "Arith"
            ro_1_line[1] = "Arith"
            ro_2_line[1] = "Arith"
            ro_3_line[1] = "Arith"
            ro_sync_line[1] = "Arith"
            
            ro_0_line[2] = "A"
            ro_1_line[2] = "A"
            ro_2_line[2] = "A"
            ro_3_line[2] = "A"
            ro_sync_line[2] = "A"
            
        if ro_0_line[0].strip() == "bsrli":
            ro_0_line[1] = "Arith"
            ro_1_line[1] = "Arith"
            ro_2_line[1] = "Arith"
            ro_3_line[1] = "Arith"
            ro_sync_line[1] = "Arith"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
        if ro_0_line[0].strip() == "bsrai":
            ro_0_line[1] = "Arith"
            ro_1_line[1] = "Arith"
            ro_2_line[1] = "Arith"
            ro_3_line[1] = "Arith"
            ro_sync_line[1] = "Arith"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
        if ro_0_line[0].strip() == "bslli":
            ro_0_line[1] = "Arith"
            ro_1_line[1] = "Arith"
            ro_2_line[1] = "Arith"
            ro_3_line[1] = "Arith"
            ro_sync_line[1] = "Arith"
            
            ro_0_line[2] = "B"
            ro_1_line[2] = "B"
            ro_2_line[2] = "B"
            ro_3_line[2] = "B"
            ro_sync_line[2] = "B"
            
print("done")

done


In [5]:
# Make the low pass filter data

lowpass_file = open("low_pass_data_" + HERTZ + "_1000.csv")

lowpassreader = csv.reader(lowpass_file)

lowpass_data = []

for row in lowpassreader:
     if ''.join(row).strip():
        lowpass_data.append(row)

for i in range(0, TRAINING_SIZE):
    ro_0_line = lowpass_data[i*5 + 0]
    ro_1_line = lowpass_data[i*5 + 1]
    ro_2_line = lowpass_data[i*5 + 2]
    ro_3_line = lowpass_data[i*5 + 3]
    ro_sync_line = lowpass_data[i*5 + 4]
    size = len(ro_sync_line)
    index = 1
    
#     print(ro_0_line)
#     print(ro_1_line)
#     print(ro_2_line)
#     print(ro_3_line)
#     print(ro_sync_line)
    
    # remove any values that do not correspond to an ro_sync = 1
    while True:
        if int(ro_sync_line[index]) == 0:
                ro_0_line.pop(index)
                ro_1_line.pop(index)
                ro_2_line.pop(index)
                ro_3_line.pop(index)
                ro_sync_line.pop(index)
                size = size - 1

        else:
            index = index + 1
                
        if index == size - 1:
            break

# trim down the size so that I don't overflow in the next
# step when I average the counts
# this range is larger than I will ever need in the training
# stage so we aren't loosing any useful information
for i in range(0, TRAINING_SIZE):
    ro_0_line = lowpass_data[i*5 + 0]
    ro_1_line = lowpass_data[i*5 + 1]
    ro_2_line = lowpass_data[i*5 + 2]
    ro_3_line = lowpass_data[i*5 + 3]
    ro_sync_line = lowpass_data[i*5 + 4]
    size = len(ro_sync_line)
    
    for j in range(size - 1, 15):
        ro_0_line.pop(index)
        ro_1_line.pop(index)
        ro_2_line.pop(index)
        ro_3_line.pop(index)
        ro_sync_line.pop(index)
            
lowpass_filter = []
print(len(lowpass_data[0]))

# for each column (i.e., clock cycle) average the counts for each sensor
for i in range(2, 16):
    s0 = 0
    s1 = 0
    s2 = 0
    s3 = 0
    for j in range(0, TRAINING_SIZE):
        s0 += int(lowpass_data[j*5 + 0][i])
        s1 += int(lowpass_data[j*5 + 1][i])
        s2 += int(lowpass_data[j*5 + 2][i])
        s3 += int(lowpass_data[j*5 + 3][i])
    s0 = int(s0/TRAINING_SIZE)
    s1 = int(s1/TRAINING_SIZE)
    s2 = int(s2/TRAINING_SIZE)
    s3 = int(s3/TRAINING_SIZE)
    lowpass_filter.append(s0)
    lowpass_filter.append(s1)
    lowpass_filter.append(s2)
    lowpass_filter.append(s3)

lowpass_filter = np.array(lowpass_filter)

print(len(lowpass_filter))
lowpass_file.close()

20
56


In [6]:
# trim all the data points that dont corresponds to a ro_sync = 1

for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
    for j in range(0, TRAINING_SIZE): # TrainingSize
        ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
        ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
        ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
        ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
        ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]
        size = len(ro_sync_line)
        # 5 correspodns to the index where the counts begin
        index = 5

        while True:
            #print(index)
            # if the index is 0 remove it and decrease the size
            # by 1 since the list size has decreased
            if int(ro_sync_line[index]) == 0:
                ro_0_line.pop(index)
                ro_1_line.pop(index)
                ro_2_line.pop(index)
                ro_3_line.pop(index)
                ro_sync_line.pop(index)
                size = size - 1
            # otherwise the index is a 1 and we are keeping it so
            # go to the next index
            else:
                index = index + 1
                
            if index == size - 1:
                break
    
print("done")

done


In [7]:
# subtract the low pass filter

for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
    for j in range(0, TRAINING_SIZE): # TrainingSize
        ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
        ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
        ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
        ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
        
        for k in range(5, 18):
            ro_0_line[k] = int(ro_0_line[k]) - lowpass_filter[(k - 5) * 4 + 0]
            ro_1_line[k] = int(ro_1_line[k]) - lowpass_filter[(k - 5) * 4 + 1]
            ro_2_line[k] = int(ro_2_line[k]) - lowpass_filter[(k - 5) * 4 + 2]
            ro_3_line[k] = int(ro_3_line[k]) - lowpass_filter[(k - 5) * 4 + 3]

# for k in range(5, 8):
#     print(lowpass_filter[(k - 5) * 4 + 0])
#     print(lowpass_filter[(k - 5) * 4 + 1])
#     print(lowpass_filter[(k - 5) * 4 + 2])
#     print(lowpass_filter[(k - 5) * 4 + 3])

print("Done")

Done


In [8]:
#  helper functions for splitting the multi-clock cycle instructions

def splitBarrel(i, j):
    ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0][:]
    ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1][:]
    ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2][:]
    ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3][:]
    ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4][:]
    
    ro_0_line[0] = "bs_1"
    ro_1_line[0] = "bs_1"
    ro_2_line[0] = "bs_1"
    ro_3_line[0] = "bs_1"
    ro_sync_line[0] = "bs_1"
    
    ro_0_line[1] = "Arith"
    ro_1_line[1] = "Arith"
    ro_2_line[1] = "Arith"
    ro_3_line[1] = "Arith"
    ro_sync_line[1] = "Arith"
    
    ro_0_line[2] = "B"
    ro_1_line[2] = "B"
    ro_2_line[2] = "B"
    ro_3_line[2] = "B"
    ro_sync_line[2] = "B"
    
    # 5 is the index of the first count
    # so removing it aligns it for the second clock cycle of the instruction
    ro_0_line.pop(5)
    ro_1_line.pop(5)
    ro_2_line.pop(5)
    ro_3_line.pop(5)
    ro_sync_line.pop(5)
    
    master_data.append(ro_0_line)
    master_data.append(ro_1_line)
    master_data.append(ro_2_line)
    master_data.append(ro_3_line)
    master_data.append(ro_sync_line)

    
    return

def splitMul(i, j):
    ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0][:]
    ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1][:]
    ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2][:]
    ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3][:]
    ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4][:]

    ro_0_line[0] = "mul_1"
    ro_1_line[0] = "mul_1"
    ro_2_line[0] = "mul_1"
    ro_3_line[0] = "mul_1"
    ro_sync_line[0] = "mul_1"

    ro_0_line[2] = "A"
    ro_1_line[2] = "A"
    ro_2_line[2] = "A"
    ro_3_line[2] = "A"
    ro_sync_line[2] = "A"

    # 5 is the index of the first count
    # so removing it aligns it for the second clock cycle of the instruction
    ro_0_line.pop(5)
    ro_1_line.pop(5)
    ro_2_line.pop(5)
    ro_3_line.pop(5)
    ro_sync_line.pop(5)

    master_data.append(ro_0_line[:])
    master_data.append(ro_1_line[:])
    master_data.append(ro_2_line[:])
    master_data.append(ro_3_line[:])
    master_data.append(ro_sync_line[:])
#     if j == 0:
#         print(master_data[Num_Of_Instructions*BLOCK_SIZE])
            
    # mul and muli are 3 clock cycle instructions so we need to do this twice
    ro_0_line[0] = "mul_2"
    ro_1_line[0] = "mul_2"
    ro_2_line[0] = "mul_2"
    ro_3_line[0] = "mul_2"
    ro_sync_line[0] = "mul_2"

    ro_0_line[2] = "A"
    ro_1_line[2] = "A"
    ro_2_line[2] = "A"
    ro_3_line[2] = "A"
    ro_sync_line[2] = "A"

    # 5 is the index of the first count
    # so removing it aligns it for the third clock cycle of the instruction
    ro_0_line.pop(5)
    ro_1_line.pop(5)
    ro_2_line.pop(5)
    ro_3_line.pop(5)
    ro_sync_line.pop(5)
        
    master_data.append(ro_0_line[:])
    master_data.append(ro_1_line[:])
    master_data.append(ro_2_line[:])
    master_data.append(ro_3_line[:])
    master_data.append(ro_sync_line[:])

    return

def splitMemStall(i, j):
    ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0][:]
    ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1][:]
    ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2][:]
    ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3][:]
    ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4][:]
    
    ro_0_line[0] = "mem_stall"
    ro_1_line[0] = "mem_stall"
    ro_2_line[0] = "mem_stall"
    ro_3_line[0] = "mem_stall"
    ro_sync_line[0] = "mem_stall"
    
    ro_0_line[1] = "Other"
    ro_1_line[1] = "Other"
    ro_2_line[1] = "Other"
    ro_3_line[1] = "Other"
    ro_sync_line[1] = "Other"
    
    ro_0_line[2] = "A"
    ro_1_line[2] = "A"
    ro_2_line[2] = "A"
    ro_3_line[2] = "A"
    ro_sync_line[2] = "A"
    
    # 5 is the index of the first count
    # so removing it aligns it for the second clock cycle of the instruction
    ro_0_line.pop(5)
    ro_1_line.pop(5)
    ro_2_line.pop(5)
    ro_3_line.pop(5)
    ro_sync_line.pop(5)
    
    master_data.append(ro_0_line)
    master_data.append(ro_1_line)
    master_data.append(ro_2_line)
    master_data.append(ro_3_line)
    master_data.append(ro_sync_line)
        
    return

print("Done")

Done


In [9]:
# Split the multi-clock cycle instructions

global Num_Of_Instructions

print(Num_Of_Instructions)
print(len(master_data))

for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
    for j in range(0, TRAINING_SIZE): # TrainingSize
        inst_name = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0][0]
        if inst_name in ["bsll", "bsrli", "bsrai", "bslli"]:
            splitBarrel(i, j)
            if(j == 0):
                Num_Of_Instructions = Num_Of_Instructions + 1
                print("1: ", inst_name)
        elif inst_name in ["mul", "muli"]:
            splitMul(i, j)
            if(j == 0):
                Num_Of_Instructions = Num_Of_Instructions + 2
                print("2: ", inst_name)
        elif inst_name in ["lhui", "lw", "lwi", "sbi", "shi", "sw", "swi"]:
            splitMemStall(i, j)
            if(j == 0):
                Num_Of_Instructions = Num_Of_Instructions + 1
                print("3: ", inst_name)
            
      # else: do nothing
print(Num_Of_Instructions)
print(len(master_data))

54
270000
3:  swi
3:  sw
3:  shi
3:  sbi
2:  muli
2:  mul
3:  lwi
3:  lw
3:  lhui
1:  bslli
1:  bsrai
1:  bsrli
1:  bsll
69
345000


In [10]:
# Change the labels for grouping 1
# i.e., combine instructions that use the same circuit within each type
# i.e., add = addc = cmp = cmpu = ...
# i.e., addi = addik = rsubk = ...
# inst, class, type, iteration, brald, add, nop, ori, ori, pre, inst, post, ....

if USEGROUP1:
    for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
        for j in range(0, TRAINING_SIZE): # TrainingSize
            ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
            ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
            ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
            ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
            ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]

            name = ro_0_line[0].strip()

            if name in ["srl", "sra"]:
                ro_0_line[0] = "shift"
                ro_1_line[0] = "shift"
                ro_2_line[0] = "shift"
                ro_3_line[0] = "shift"
                ro_sync_line[0] = "shift"

                ro_0_line[1] = "Logical"
                ro_1_line[1] = "Logical"
                ro_2_line[1] = "Logical"
                ro_3_line[1] = "Logical"
                ro_sync_line[1] = "Logical"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["bsll", "bsrli", "bsrai", "bslli", "bs_1"]:
                ro_0_line[0] = "barrell_shift"
                ro_1_line[0] = "barrell_shift"
                ro_2_line[0] = "barrell_shift"
                ro_3_line[0] = "barrell_shift"
                ro_sync_line[0] = "barrell_shift"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["mul", "mul_1", "mul_2"]:
                ro_0_line[0] = "mul"
                ro_1_line[0] = "mul"
                ro_2_line[0] = "mul"
                ro_3_line[0] = "mul"
                ro_sync_line[0] = "mul"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["beqi", "beqid", "bgei", "bgeid", "bgti", 
                        "bgtid", "blei", "bleid", "blti", "bltid", "bnei", "bneid"]:
                ro_0_line[0] = "conditional"
                ro_1_line[0] = "conditional"
                ro_2_line[0] = "conditional"
                ro_3_line[0] = "conditional"
                ro_sync_line[0] = "conditional"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "C"
                ro_1_line[2] = "C"
                ro_2_line[2] = "C"
                ro_3_line[2] = "C"
                ro_sync_line[2] = "C"

            if name in ["bra", "brald", "bri", "brai", "brid", "brlid"]:
                ro_0_line[0] = "unconditional"
                ro_1_line[0] = "unconditional"
                ro_2_line[0] = "unconditional"
                ro_3_line[0] = "unconditional"
                ro_sync_line[0] = "unconditional"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "U"
                ro_1_line[2] = "U"
                ro_2_line[2] = "U"
                ro_3_line[2] = "U"
                ro_sync_line[2] = "U"

            if name in ["rtid", "rted", "rtsd"]:
                ro_0_line[0] = "conditional"
                ro_1_line[0] = "return"
                ro_2_line[0] = "return"
                ro_3_line[0] = "return"
                ro_sync_line[0] = "return"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "U"
                ro_1_line[2] = "U"
                ro_2_line[2] = "U"
                ro_3_line[2] = "U"
                ro_sync_line[2] = "U"

            if name in ["lhui", "lwi"]:
                ro_0_line[0] = "load_i"
                ro_1_line[0] = "load_i"
                ro_2_line[0] = "load_i"
                ro_3_line[0] = "load_i"
                ro_sync_line[0] = "load_i"

                ro_0_line[1] = "Memory"
                ro_1_line[1] = "Memory"
                ro_2_line[1] = "Memory"
                ro_3_line[1] = "Memory"
                ro_sync_line[1] = "Memory"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["sbi", "shi", "swi"]:
                ro_0_line[0] = "store_i"
                ro_1_line[0] = "store_i"
                ro_2_line[0] = "store_i"
                ro_3_line[0] = "store_i"
                ro_sync_line[0] = "store_i"

                ro_0_line[1] = "Memory"
                ro_1_line[1] = "Memory"
                ro_2_line[1] = "Memory"
                ro_3_line[1] = "Memory"
                ro_sync_line[1] = "Memory"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["andi", "ori", "xori"]:
                ro_0_line[0] = "logic_i"
                ro_1_line[0] = "logic_i"
                ro_2_line[0] = "logic_i"
                ro_3_line[0] = "logic_i"
                ro_sync_line[0] = "logic_i"

                ro_0_line[1] = "Logical"
                ro_1_line[1] = "Logical"
                ro_2_line[1] = "Logical"
                ro_3_line[1] = "Logical"
                ro_sync_line[1] = "Logical"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["and", "or", "xor"]:
                ro_0_line[0] = "logic"
                ro_1_line[0] = "logic"
                ro_2_line[0] = "logic"
                ro_3_line[0] = "logic"
                ro_sync_line[0] = "logic"

                ro_0_line[1] = "Logical"
                ro_1_line[1] = "Logical"
                ro_2_line[1] = "Logical"
                ro_3_line[1] = "Logical"
                ro_sync_line[1] = "Logical"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["addi", "addik", "rsubi"]:
                ro_0_line[0] = "add_i"
                ro_1_line[0] = "add_i"
                ro_2_line[0] = "add_i"
                ro_3_line[0] = "add_i"
                ro_sync_line[0] = "add_i"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["add", "addk", "rsub", "addc", "cmp", "cmpu", "rsubk"]:
                ro_0_line[0] = "add"
                ro_1_line[0] = "add"
                ro_2_line[0] = "add"
                ro_3_line[0] = "add"
                ro_sync_line[0] = "add"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"
print("done")

In [11]:
# Change the labels for grouping 2
# inst, class, type, iteration, brald, add, nop, ori, ori, pre, inst, post, ....

# i.e., combine instructions that use the same circuit ignoring the type
# i.e., add = addc = cmp = cmpu = ... = addi = addik = rsubk = ...
# you will need to comment all of the non-instonly models at the end or the program will crash

if USEGROUP2:
    for i in range(0, INSTRUCTIION_COUNT): # NumOfInstructions
        for j in range(0, TRAINING_SIZE): # TrainingSize
            ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
            ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
            ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
            ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
            ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]

            name = ro_0_line[0].strip()

            if name in ["srl", "sra"]:
                ro_0_line[0] = "shift"
                ro_1_line[0] = "shift"
                ro_2_line[0] = "shift"
                ro_3_line[0] = "shift"
                ro_sync_line[0] = "shift"

                ro_0_line[1] = "Logical"
                ro_1_line[1] = "Logical"
                ro_2_line[1] = "Logical"
                ro_3_line[1] = "Logical"
                ro_sync_line[1] = "Logical"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["bsll", "bsrli", "bsrai", "bslli", "bs_1"]:
                ro_0_line[0] = "barrell_shift"
                ro_1_line[0] = "barrell_shift"
                ro_2_line[0] = "barrell_shift"
                ro_3_line[0] = "barrell_shift"
                ro_sync_line[0] = "barrell_shift"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "B"
                ro_1_line[2] = "B"
                ro_2_line[2] = "B"
                ro_3_line[2] = "B"
                ro_sync_line[2] = "B"

            if name in ["mul", "mul_1", "mul_2", "muli"]:
                ro_0_line[0] = "mul"
                ro_1_line[0] = "mul"
                ro_2_line[0] = "mul"
                ro_3_line[0] = "mul"
                ro_sync_line[0] = "mul"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["beqi", "beqid", "bgei", "bgeid", "bgti", 
                        "bgtid", "blei", "bleid", "blti", "bltid", "bnei", "bneid"]:
                ro_0_line[0] = "conditional"
                ro_1_line[0] = "conditional"
                ro_2_line[0] = "conditional"
                ro_3_line[0] = "conditional"
                ro_sync_line[0] = "conditional"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "C"
                ro_1_line[2] = "C"
                ro_2_line[2] = "C"
                ro_3_line[2] = "C"
                ro_sync_line[2] = "C"

            if name in ["bra", "brald", "bri", "brai", "brid", "brlid"]:
                ro_0_line[0] = "unconditional"
                ro_1_line[0] = "unconditional"
                ro_2_line[0] = "unconditional"
                ro_3_line[0] = "unconditional"
                ro_sync_line[0] = "unconditional"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "U"
                ro_1_line[2] = "U"
                ro_2_line[2] = "U"
                ro_3_line[2] = "U"
                ro_sync_line[2] = "U"

            if name in ["rtid", "rted", "rtsd"]:
                ro_0_line[0] = "conditional"
                ro_1_line[0] = "return"
                ro_2_line[0] = "return"
                ro_3_line[0] = "return"
                ro_sync_line[0] = "return"

                ro_0_line[1] = "Branching"
                ro_1_line[1] = "Branching"
                ro_2_line[1] = "Branching"
                ro_3_line[1] = "Branching"
                ro_sync_line[1] = "Branching"

                ro_0_line[2] = "U"
                ro_1_line[2] = "U"
                ro_2_line[2] = "U"
                ro_3_line[2] = "U"
                ro_sync_line[2] = "U"

            if name in ["lhui", "lwi", "lw"]:
                ro_0_line[0] = "load"
                ro_1_line[0] = "load"
                ro_2_line[0] = "load"
                ro_3_line[0] = "load"
                ro_sync_line[0] = "load"

                ro_0_line[1] = "Memory"
                ro_1_line[1] = "Memory"
                ro_2_line[1] = "Memory"
                ro_3_line[1] = "Memory"
                ro_sync_line[1] = "Memory"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["sbi", "shi", "swi", "sw"]:
                ro_0_line[0] = "store"
                ro_1_line[0] = "store"
                ro_2_line[0] = "store"
                ro_3_line[0] = "store"
                ro_sync_line[0] = "store"

                ro_0_line[1] = "Memory"
                ro_1_line[1] = "Memory"
                ro_2_line[1] = "Memory"
                ro_3_line[1] = "Memory"
                ro_sync_line[1] = "Memory"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["and", "or", "xor", "andi", "ori", "xori"]:
                ro_0_line[0] = "logic"
                ro_1_line[0] = "logic"
                ro_2_line[0] = "logic"
                ro_3_line[0] = "logic"
                ro_sync_line[0] = "logic"

                ro_0_line[1] = "Logical"
                ro_1_line[1] = "Logical"
                ro_2_line[1] = "Logical"
                ro_3_line[1] = "Logical"
                ro_sync_line[1] = "Logical"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"

            if name in ["add", "addk", "rsub", "addc", "cmp", "cmpu", "rsubk",
                       "addi", "addik", "rsubi"]:
                ro_0_line[0] = "add"
                ro_1_line[0] = "add"
                ro_2_line[0] = "add"
                ro_3_line[0] = "add"
                ro_sync_line[0] = "add"

                ro_0_line[1] = "Arith"
                ro_1_line[1] = "Arith"
                ro_2_line[1] = "Arith"
                ro_3_line[1] = "Arith"
                ro_sync_line[1] = "Arith"

                ro_0_line[2] = "A"
                ro_1_line[2] = "A"
                ro_2_line[2] = "A"
                ro_3_line[2] = "A"
                ro_sync_line[2] = "A"
print("done")

In [12]:
# trim the count portion of the data to three points
# inst, class, type, iteration, brald, add, nop, ori, ori, pre, inst, post, ....
#    0,      1,   2,         3,     4,   5,   6,   7,   8,   9,  10,    11, ....

for i in range(0, Num_Of_Instructions): # NumOfInstructions
    print(str(i) + ": " + master_data[i*BLOCK_SIZE][0])
    for j in range(0, TRAINING_SIZE): # TrainingSize
        ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
        ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
        ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
        ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
        ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]
    
        size = len(ro_0_line)
        
        # size -1 , 12, -1
        for k in range(size - 1, 12, -1):
            ro_0_line.pop(k)
            ro_1_line.pop(k)
            ro_2_line.pop(k)
            ro_3_line.pop(k)
            ro_sync_line.pop(k)
            
        # 9, 4, -1
        for k in range(9, 4, -1):
#             print(k)
#             print(ro_0_line)
            ro_0_line.pop(k)
            ro_1_line.pop(k)
            ro_2_line.pop(k)
            ro_3_line.pop(k)
            ro_sync_line.pop(k)

print("Done")

0: xori
1: xor
2: swi
3: sw
4: srl
5: sra
6: shi
7: sext8
8: sbi
9: rtsd
10: rted
11: rtid
12: rsubi
13: rsubk
14: rsub
15: ori
16: or
17: muli
18: mul
19: lwi
20: lw
21: lhui
22: imm
23: cmp
24: cmpu
25: bslli
26: bsrai
27: bsrli
28: bsll
29: brlid
30: brid
31: brai
32: bri
33: brald
34: bra
35: bneid
36: bnei
37: bltid
38: blti
39: bleid
40: blei
41: bgtid
42: bgti
43: bgeid
44: bgei
45: beqid
46: beqi
47: andi
48: and
49: addik
50: addi
51: addk
52: addc
53: add
54: mem_stall
55: mem_stall
56: mem_stall
57: mem_stall
58: mul_1
59: mul_1
60: mul_1
61: mul_1
62: mem_stall
63: mem_stall
64: mem_stall
65: bs_1
66: bs_1
67: bs_1
68: bs_1
Done


In [13]:
# just for troubleshooting
# print(master_data[57*BLOCK_SIZE])
# print(master_data[58*BLOCK_SIZE])
# print(master_data[59*BLOCK_SIZE])
# print(master_data[60*BLOCK_SIZE])
# print(master_data[61*BLOCK_SIZE])
# print(master_data[62*BLOCK_SIZE])
print(master_data[4])

['xori', ' Logical', ' B', ' 0', ' sync', ' 1', ' 1', ' 1']


In [14]:
# Split the data into sets
# i.e., inst, class, type, iteration, count_0 = (count_0_0, count_0_1, count_0_2, count_0_3), count_1, count_2

master_data_new = []


for i in range(0, Num_Of_Instructions): # NumOfInstructions
    for j in range(0, TRAINING_SIZE): # TrainingSize
        count_0 = []
        count_1 = []
        count_2 = []
        count_3 = []
        line = []
        ro_0_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 0]
        ro_1_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 1]
        ro_2_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 2]
        ro_3_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 3]
        #ro_sync_line = master_data[i*BLOCK_SIZE + j*SENSOR_COUNT + 4]
        
        line.append(ro_0_line[0]) # inst
        line.append(ro_0_line[1]) # class
        line.append(ro_0_line[2]) # type
        line.append(ro_0_line[3]) # iteration

        count_0.append(ro_0_line[5])
        count_0.append(ro_0_line[6])
        count_0.append(ro_0_line[7])
#         count_0.append(ro_0_line[8])
#         count_0.append(ro_0_line[9])
        
        count_1.append(ro_1_line[5])
        count_1.append(ro_1_line[6])
        count_1.append(ro_1_line[7])
#         count_1.append(ro_1_line[8])
#         count_1.append(ro_1_line[9])
        
        count_2.append(ro_2_line[5])
        count_2.append(ro_2_line[6])
        count_2.append(ro_2_line[7])
#         count_2.append(ro_2_line[8])
#         count_2.append(ro_2_line[9])
        
        count_3.append(ro_3_line[5])
        count_3.append(ro_3_line[6])
        count_3.append(ro_3_line[7])
#         count_3.append(ro_3_line[8])
#         count_3.append(ro_3_line[9])
        
        line.append(count_0)
        line.append(count_1)
        line.append(count_2)
        line.append(count_3)
        
        master_data_new.append(line)

print(len(master_data_new))

69000


In [15]:
# just for troubleshooting
# print("hello")
# print(len(master_data_new))
# print(master_data_new[0])

# for i in range(0, Num_Of_Instructions): # NumOfInstructions
#     for j in range(0, 2): # TrainingSize
#         print(master_data_new[i*TRAINING_SIZE + j])


In [16]:
# create the 14 models
# top level
class_model = DecisionTreeClassifier()

# class level
arithmetic_type_model = DecisionTreeClassifier()
logical_type_model = DecisionTreeClassifier()
branching_type_model = DecisionTreeClassifier()
memaccess_type_model = DecisionTreeClassifier()

# type level
arithmetic_a_model = DecisionTreeClassifier()
arithmetic_b_model = DecisionTreeClassifier()
logical_a_model = DecisionTreeClassifier()
logical_b_model = DecisionTreeClassifier()
branching_c_model = DecisionTreeClassifier()
branching_u_model = DecisionTreeClassifier()
memaccess_a_model = DecisionTreeClassifier()
memaccess_b_model = DecisionTreeClassifier()
other_model = DecisionTreeClassifier()

# Extra models
# 1-layer for inst only
inst_only_model = DecisionTreeClassifier()

In [17]:
# Make an 80/20 split for training/testing
# inst, class, type, iteration, count_0 = (count_0_0, count_0_1, count_0_2, count_0_3), count_1, count_2

x = np.array([i[4:8] for i in master_data_new])
y = np.array([i[0:4] for i in master_data_new])

# print(master_data_new[1])
# print(x[1])
# print(y[1])
# print(len(x))
# print(len(y))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
x_train = np.array(x_train)
x_test  = np.array(x_test)
y_train  = np.array(y_train)
y_test = np.array(y_test)

# reshake the counts so they can be feed into the decesion tree
nsample, nx, ny = x_train.shape
x_train_2d = x_train.reshape((nsample, nx*ny))

nsample, nx, ny = x_test.shape
x_test_2d = x_test.reshape((nsample, nx*ny))

# create arrays for the traing labels
y_train_inst = np.array([i[0] for i in y_train])
y_train_class = np.array([i[1] for i in y_train])
y_train_type = np.array([i[2] for i in y_train])

# create arrays for the testing labels
y_test_class = np.array([i[1] for i in y_test])
y_test_inst = np.array([i[0] for i in y_test])
y_test_type = np.array([i[2] for i in y_test])

print("done")

done


In [18]:
# just for troubleshooting
print(x_train_2d[1])
print(y_train_inst)
print(x_train_2d)

[ 0  0  0  0 -1  0  1  1  1  1  0  1]
['addi' 'swi' 'rsubk' ... 'bsrai' 'sbi' 'bra']
[[  0   0   1 ...  32 -32  32]
 [  0   0   0 ...   1   0   1]
 [  0   1   1 ...  32 -31  33]
 ...
 [  0   1   1 ...   1   0   1]
 [  0   0   0 ...  32 -31  33]
 [  1   0   0 ...   0   0   1]]


In [19]:
# build the 14 training data sets
arith_data = []
arith_a_data = []
arith_b_data = []
logical_data = []
logical_a_data = []
logical_b_data = []
branching_data = []
branching_u_data = []
branching_c_data = []
memory_data = []
memory_a_data = []
memory_b_data = []
other_data = []

arith_data_label = []
arith_a_data_label = []
arith_b_data_label = []
logical_data_label = []
logical_a_data_label = []
logical_b_data_label = []
branching_data_label = []
branching_u_data_label = []
branching_c_data_label = []
memory_data_label = []
memory_a_data_label = []
memory_b_data_label = []
other_data_label = []

# Build their label datasets

for i in range(0, len(y_train_class)):
    if y_train_class[i].strip() == "Arith":
        arith_data.append(x_train_2d[i])
        arith_data_label.append(y_train_type[i])
        if y_train_type[i].strip() == "A":
            arith_a_data.append(x_train_2d[i])
            arith_a_data_label.append(y_train_inst[i])
        elif y_train_type[i].strip() == "B":
            arith_b_data.append(x_train_2d[i])
            arith_b_data_label.append(y_train_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_train_type[i])
            
    elif y_train_class[i].strip() == "Logical":
        logical_data.append(x_train_2d[i])
        logical_data_label.append(y_train_type[i])
        if y_train_type[i].strip() == "A":
            logical_a_data.append(x_train_2d[i])
            logical_a_data_label.append(y_train_inst[i])
        elif y_train_type[i].strip() == "B":
            logical_b_data.append(x_train_2d[i])
            logical_b_data_label.append(y_train_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_train_type[i])
                    
    elif y_train_class[i].strip() == "Branching":
        branching_data.append(x_train_2d[i])
        branching_data_label.append(y_train_type[i])
        if y_train_type[i].strip() == "U":
            branching_u_data.append(x_train_2d[i])
            branching_u_data_label.append(y_train_inst[i])
        elif y_train_type[i].strip() == "C":
            branching_c_data.append(x_train_2d[i])
            branching_c_data_label.append(y_train_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_train_type[i])
                
    elif y_train_class[i].strip() == "Memory":
        memory_data.append(x_train_2d[i])
        memory_data_label.append(y_train_type[i])
        if y_train_type[i].strip() == "A":
            memory_a_data.append(x_train_2d[i])
            memory_a_data_label.append(y_train_inst[i])
        elif y_train_type[i].strip() == "B":
            memory_b_data.append(x_train_2d[i])
            memory_b_data_label.append(y_train_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_train_type[i])
                
    elif y_train_class[i].strip() == "Other":
        other_data.append(x_train_2d[i])
        other_data_label.append(y_train_inst[i])
    else:
        print("Error! Invalid Class Label: ", y_train_class[i])
        print(i)

# convery to arrays        
arith_data = np.array(arith_data)
arith_a_data = np.array(arith_a_data)
arith_b_data = np.array(arith_b_data)
logical_data = np.array(logical_data)
logical_a_data = np.array(logical_a_data)
logical_b_data = np.array(logical_b_data)
branching_data = np.array(branching_data)
branching_u_data = np.array(branching_u_data)
branching_c_data = np.array(branching_c_data)
memory_data = np.array(memory_data)
memory_a_data = np.array(memory_a_data)
memory_b_data = np.array(memory_b_data)
other_data = np.array(other_data)

arith_data_label = np.array(arith_data_label)
arith_a_data_label = np.array(arith_a_data_label)
arith_b_data_label = np.array(arith_b_data_label)
logical_data_label = np.array(logical_data_label)
logical_a_data_label = np.array(logical_a_data_label)
logical_b_data_label = np.array(logical_b_data_label)
branching_data_label = np.array(branching_data_label)
branching_u_data_label = np.array(branching_u_data_label)
branching_c_data_label = np.array(branching_c_data_label)
memory_data_label = np.array(memory_data_label)
memory_a_data_label = np.array(memory_a_data_label)
memory_b_data_label = np.array(memory_b_data_label)
other_data_label = np.array(other_data_label)


print("Done")

Done


In [20]:
# build the 14 testing data sets
arith_data_test = []
arith_a_data_test = []
arith_b_data_test = []
logical_data_test = []
logical_a_data_test = []
logical_b_data_test = []
branching_data_test = []
branching_u_data_test = []
branching_c_data_test = []
memory_data_test = []
memory_a_data_test = []
memory_b_data_test = []
other_data_test = []

arith_data_test_label = []
arith_a_data_test_label = []
arith_b_data_test_label = []
logical_data_test_label = []
logical_a_data_test_label = []
logical_b_data_test_label = []
branching_data_test_label = []
branching_u_data_test_label = []
branching_c_data_test_label = []
memory_data_test_label = []
memory_a_data_test_label = []
memory_b_data_test_label = []
other_data_test_label = []

# Build their label datasets

for i in range(0, len(y_test_class)):
    if y_test_class[i].strip() == "Arith":
        arith_data_test.append(x_test_2d[i])
        arith_data_test_label.append(y_test_type[i])
        if y_test_type[i].strip() == "A":
            arith_a_data_test.append(x_test_2d[i])
            arith_a_data_test_label.append(y_test_inst[i])
        elif y_test_type[i].strip() == "B":
            arith_b_data_test.append(x_test_2d[i])
            arith_b_data_test_label.append(y_test_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_test_type[i])
            
    elif y_test_class[i].strip() == "Logical":
        logical_data_test.append(x_test_2d[i])
        logical_data_test_label.append(y_test_type[i])
        if y_test_type[i].strip() == "A":
            logical_a_data_test.append(x_test_2d[i])
            logical_a_data_test_label.append(y_test_inst[i])
        elif y_test_type[i].strip() == "B":
            logical_b_data_test.append(x_test_2d[i])
            logical_b_data_test_label.append(y_test_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_test_type[i])
                    
    elif y_test_class[i].strip() == "Branching":
        branching_data_test.append(x_test_2d[i])
        branching_data_test_label.append(y_test_type[i])
        if y_test_type[i].strip() == "U":
            branching_u_data_test.append(x_test_2d[i])
            branching_u_data_test_label.append(y_test_inst[i])
        elif y_test_type[i].strip() == "C":
            branching_c_data_test.append(x_test_2d[i])
            branching_c_data_test_label.append(y_test_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_test_type[i])
                
    elif y_test_class[i].strip() == "Memory":
        memory_data_test.append(x_test_2d[i])
        memory_data_test_label.append(y_test_type[i])
        if y_test_type[i].strip() == "A":
            memory_a_data_test.append(x_test_2d[i])
            memory_a_data_test_label.append(y_test_inst[i])
        elif y_test_type[i].strip() == "B":
            memory_b_data_test.append(x_test_2d[i])
            memory_b_data_test_label.append(y_test_inst[i])
        else:
            print("Error! Invalid Type Label: ", y_test_type[i])
                
    elif y_test_class[i].strip() == "Other":
        other_data_test.append(x_test_2d[i])
        other_data_test_label.append(y_test_inst[i])
    else:
        print("Error! Invalid Class Label: ", y_test_class[i])
        print(i)

# convery to arrays        
arith_data_test = np.array(arith_data_test)
arith_a_data_test = np.array(arith_a_data_test)
arith_b_data_test = np.array(arith_b_data_test)
logical_data_test = np.array(logical_data_test)
logical_a_data_test = np.array(logical_a_data_test)
logical_b_data_test = np.array(logical_b_data_test)
branching_data_test = np.array(branching_data_test)
branching_u_data_test = np.array(branching_u_data_test)
branching_c_data_test = np.array(branching_c_data_test)
memory_data_test = np.array(memory_data_test)
memory_a_data_test = np.array(memory_a_data_test)
memory_b_data_test = np.array(memory_b_data_test)
other_data_test = np.array(other_data_test)

arith_data_test_label = np.array(arith_data_test_label)
arith_a_data_test_label = np.array(arith_a_data_test_label)
arith_b_data_test_label = np.array(arith_b_data_test_label)
logical_data_test_label = np.array(logical_data_test_label)
logical_a_data_test_label = np.array(logical_a_data_test_label)
logical_b_data_test_label = np.array(logical_b_data_test_label)
branching_data_test_label = np.array(branching_data_test_label)
branching_u_data_test_label = np.array(branching_u_data_test_label)
branching_c_data_test_label = np.array(branching_c_data_test_label)
memory_data_test_label = np.array(memory_data_test_label)
memory_a_data_test_label = np.array(memory_a_data_test_label)
memory_b_data_test_label = np.array(memory_b_data_test_label)
other_data_test_label = np.array(other_data_test_label)


print("Done")

Done


In [21]:
# train the models
# top level
# inst, class, type, iteration, count_0 = (count_0_0, count_0_1, count_0_2, count_0_3), count_1, count_2
# if you are using group2 then comment out everything except for the inst_only_model

# extra models
inst_only_model = inst_only_model.fit(x_train_2d, y_train_inst)

# top level
class_model = class_model.fit(x_train_2d, y_train_class)

# class level
arithmetic_type_model = arithmetic_type_model.fit(arith_data, arith_data_label)
logical_type_model = logical_type_model.fit(logical_data, logical_data_label)
branching_type_model = branching_type_model.fit(branching_data, branching_data_label)
memaccess_type_model = memaccess_type_model.fit(memory_data, memory_data_label)

# type level
arithmetic_a_model = arithmetic_a_model.fit(arith_a_data, arith_a_data_label)
arithmetic_b_model = arithmetic_b_model.fit(arith_b_data, arith_b_data_label)
logical_a_model = logical_a_model.fit(logical_a_data, logical_a_data_label)
logical_b_model = logical_b_model.fit(logical_b_data, logical_b_data_label)
branching_c_model = branching_c_model.fit(branching_c_data, branching_c_data_label)
branching_u_model = branching_u_model.fit(branching_u_data, branching_u_data_label)
memaccess_a_model = memaccess_a_model.fit(memory_a_data, memory_a_data_label)
memaccess_b_model = memaccess_b_model.fit(memory_b_data, memory_b_data_label)
other_model = other_model.fit(other_data, other_data_label)

print("done")

done


In [22]:
# evaluate the models individually
#Predict the response for test dataset
# if you are using group2 then comment out everything except for the inst_only_model

y_pred_inst_only = inst_only_model.predict(x_test_2d)
y_pred_class = class_model.predict(x_test_2d)

# class level
y_pred_arith = arithmetic_type_model.predict(arith_data_test)
y_pred_logical = logical_type_model.predict(logical_data_test)
y_pred_branching = branching_type_model.predict(branching_data_test)
y_pred_memaccess = memaccess_type_model.predict(memory_data_test)

# type level
y_pred_arith_a = arithmetic_a_model.predict(arith_a_data_test)
y_pred_arith_b = arithmetic_b_model.predict(arith_b_data_test)
y_pred_log_a = logical_a_model.predict(logical_a_data_test)
y_pred_log_b = logical_b_model.predict(logical_b_data_test)
y_pred_branch_c = branching_c_model.predict(branching_c_data_test)
y_pred_branch_u = branching_u_model.predict(branching_u_data_test)
y_pred_mem_a = memaccess_a_model.predict(memory_a_data_test)
y_pred_mem_b = memaccess_b_model.predict(memory_b_data_test)
y_pred_other = other_model.predict(other_data_test)

# Model Accuracy, how often is the classifier correct?
print("Class Accuracy:", metrics.accuracy_score(y_test_class, y_pred_class))
print("Inst only Accuracy:", metrics.accuracy_score(y_test_inst, y_pred_inst_only))

print("Arith Accuracy:", metrics.accuracy_score(arith_data_test_label, y_pred_arith))
print("Logical Accuracy:", metrics.accuracy_score(logical_data_test_label, y_pred_logical))
print("Branching Accuracy:", metrics.accuracy_score(branching_data_test_label, y_pred_branching))
print("Memory Access Accuracy:", metrics.accuracy_score(memory_data_test_label, y_pred_memaccess))

print("Arith A Accuracy:", metrics.accuracy_score(arith_a_data_test_label, y_pred_arith_a))
print("Arith B Accuracy:", metrics.accuracy_score(arith_b_data_test_label, y_pred_arith_b))
print("Logical A Accuracy:", metrics.accuracy_score(logical_a_data_test_label, y_pred_log_a))
print("Logical B Accuracy:", metrics.accuracy_score(logical_b_data_test_label, y_pred_log_b))
print("Branching C Accuracy:", metrics.accuracy_score(branching_c_data_test_label, y_pred_branch_c))
print("Branching U Accuracy:", metrics.accuracy_score(branching_u_data_test_label, y_pred_branch_u))
print("Memory A Accuracy:", metrics.accuracy_score(memory_a_data_test_label, y_pred_mem_a))
print("Memory B Accuracy:", metrics.accuracy_score(memory_b_data_test_label, y_pred_mem_b))
print("Other Accuracy:", metrics.accuracy_score(other_data_test_label, y_pred_other))

Class Accuracy: 0.42130434782608694
Inst only Accuracy: 0.15405797101449276
Arith Accuracy: 0.5621838111298483
Logical Accuracy: 0.3363329583802025
Branching Accuracy: 0.562573929500828
Memory Access Accuracy: 0.6569241475295755
Arith A Accuracy: 0.4652923853596971
Arith B Accuracy: 0.4719053654414871
Logical A Accuracy: 0.36649214659685864
Logical B Accuracy: 0.18423236514522823
Branching C Accuracy: 0.11993450675399099
Branching U Accuracy: 0.1491031390134529
Memory A Accuracy: 0.5403587443946188
Memory B Accuracy: 0.26437941473259335
Other Accuracy: 1.0


In [23]:
# Evaluate the combined model's accuracy on the test data
# x_test_2d y_test_inst
# y_test_class
# y_test_inst
# y_test_type

# Top level
final_pred = []
top_actual = []
top_pred = []

# Type level
arith_pred = []
arith_actual = []
logical_pred = []
logical_actual = []
branching_pred = []
branching_actual = []
memory_pred = []
memory_actual = []

# Inst level
arith_a_pred = []
arith_a_actual = []
arith_b_pred = []
arith_b_actual = []
logical_a_pred = []
logical_a_actual = []
logical_b_pred = []
logical_b_actual = []
branching_c_pred = []
branching_c_actual = []
branching_u_pred = []
branching_u_actual = []
memory_a_pred = []
memory_a_actual = []
memory_b_pred = []
memory_b_actual = []
other_pred = []
other_actual = []

# Type level
arith_node_pred = []
arith_node_actual = []
logical_node_pred = []
logical_node_actual = []
branching_node_pred = []
branching_node_actual = []
memory_node_pred = []
memory_node_actual = []

# Inst level
arith_node_a_pred = []
arith_node_a_actual = []
arith_node_b_pred = []
arith_node_b_actual = []
logical_node_a_pred = []
logical_node_a_actual = []
logical_node_b_pred = []
logical_node_b_actual = []
branching_node_c_pred = []
branching_node_c_actual = []
branching_node_u_pred = []
branching_node_u_actual = []
memory_node_a_pred = []
memory_node_a_actual = []
memory_node_b_pred = []
memory_node_b_actual = []
other_node_pred = []
other_node_actual = []

for i in range(0, len(x_test_2d)):
    data = x_test_2d[i]
    data = data.reshape(1, -1)
    class_pred = class_model.predict(data)
    top_actual.append(y_test_class[i])
    top_pred.append(class_pred)
    
    if class_pred[0].strip() ==  "Arith":
        arith_actual.append(y_test_class[i].strip())
        arith_pred.append(class_pred[0].strip())
        type_pred = arithmetic_type_model.predict(data)
        arith_node_pred.append(type_pred[0].strip())
        if y_test_class[i].strip() == "Arith":
            arith_node_actual.append(y_test_type[i].strip())
        else:
            # this is for a type that will always correspond to wrong.
            # since the arith model can't accurately predict a non
            # arith type, we need to make sure that is will always be wrong
            # i.e., we don't want it to think is correct
            # if it assins a logic-a instruction to the a group here
            arith_node_actual.append("z")
        if type_pred[0].strip() == "A":
            inst_pred = arithmetic_a_model.predict(data)
            arith_a_actual.append(y_test_type[i].strip())
            arith_a_pred.append(type_pred[0].strip())
            arith_node_a_pred.append(inst_pred[0].strip())
            arith_node_a_actual.append(y_test_inst[i].strip())
        elif type_pred[0].strip() == "B":
            inst_pred = arithmetic_b_model.predict(data)
            arith_b_actual.append(y_test_type[i].strip())
            arith_b_pred.append(type_pred[0].strip())
            arith_node_b_pred.append(inst_pred[0].strip())
            arith_node_b_actual.append(y_test_inst[i].strip())            
        else:
            print("Error: Invalid Arithmetic Type prediction")
            break
    elif class_pred[0].strip() == "Logical":
        type_pred = logical_type_model.predict(data)
        logical_actual.append(y_test_class[i].strip())
        logical_pred.append(class_pred[0].strip())
        logical_node_pred.append(type_pred[0].strip())
        if y_test_class[i].strip() == "Logical":
            logical_node_actual.append(y_test_type[i].strip())
        else:
            # see comment for arith branch for explenation
            logical_node_actual.append("z")
        if type_pred[0].strip() == "A":
            inst_pred = logical_a_model.predict(data)
            logical_a_actual.append(y_test_type[i].strip())
            logical_a_pred.append(type_pred[0].strip())
            logical_node_a_pred.append(inst_pred[0].strip())
            logical_node_a_actual.append(y_test_inst[i].strip())            
        elif type_pred[0].strip() == "B":
            inst_pred = logical_b_model.predict(data)
            logical_b_actual.append(y_test_type[i].strip())
            logical_b_pred.append(type_pred[0].strip())
            logical_node_b_pred.append(inst_pred[0].strip())
            logical_node_b_actual.append(y_test_inst[i].strip())            
        else:
            print("Error: Invalid Arithmetic Type prediction")
            break
    elif class_pred[0].strip() == "Branching":
        type_pred = branching_type_model.predict(data)
        branching_actual.append(y_test_class[i].strip())
        branching_pred.append(class_pred[0].strip())
        branching_node_pred.append(type_pred[0].strip())
        branching_node_actual.append(y_test_type[i].strip())
        if type_pred[0].strip() == "C":
            inst_pred = branching_c_model.predict(data)
            branching_c_actual.append(y_test_type[i].strip())
            branching_c_pred.append(type_pred[0].strip())
            branching_node_c_pred.append(inst_pred[0].strip())
            branching_node_c_actual.append(y_test_inst[i].strip())            
        elif type_pred[0].strip() == "U":
            inst_pred = branching_u_model.predict(data)
            branching_u_actual.append(y_test_type[i].strip())
            branching_u_pred.append(type_pred[0].strip())
            branching_node_u_pred.append(inst_pred[0].strip())
            branching_node_u_actual.append(y_test_inst[i].strip())            
        else:
            print("Error: Invalid Arithmetic Type prediction")
            break
    elif class_pred[0].strip() == "Memory":
        type_pred = memaccess_type_model.predict(data)
        memory_actual.append(y_test_class[i].strip())
        memory_pred.append(class_pred[0].strip())
        memory_node_pred.append(type_pred[0].strip())
        if y_test_class[i].strip() == "Memory":
            memory_node_actual.append(y_test_type[i].strip())
        else:
            # see comment for arith branch for explenation
            memory_node_actual.append("z")
        if type_pred[0].strip() == "A":
            inst_pred = memaccess_a_model.predict(data)
            memory_a_actual.append(y_test_type[i].strip())
            memory_a_pred.append(type_pred[0].strip())
            memory_node_a_pred.append(inst_pred[0].strip())
            memory_node_a_actual.append(y_test_inst[i].strip())            
        elif type_pred[0].strip() == "B":
            inst_pred = memaccess_b_model.predict(data)
            memory_b_actual.append(y_test_type[i].strip())
            memory_b_pred.append(type_pred[0].strip())
            memory_node_b_pred.append(inst_pred[0].strip())
            memory_node_b_actual.append(y_test_inst[i].strip())            
        else:
            print("Error: Invalid Arithmetic Type prediction")
            break
    elif class_pred[0].strip() == "Other":
        inst_pred = other_model.predict(other_data_test)
        other_actual.append(y_test_class[i].strip())
        other_pred.append(class_pred[0].strip())
        other_node_pred.append(inst_pred[0].strip())
        other_node_actual.append(y_test_inst[i].strip())
    else:
        print("Error: Invalid Class prediction")
        print(class_pred)
        break
    
    
    # add this prediction to the pred array
    final_pred.append(inst_pred[0].strip())


pred = np.array(final_pred)
top_actual = np.array(top_actual)
top_pred = np.array(top_pred)
arith_pred = np.array(arith_pred)
arith_actual = np.array(arith_actual)
logical_pred = np.array(logical_pred)
logical_actual = np.array(logical_actual)
branching_pred = np.array(branching_pred)
branching_actual = np.array(branching_actual)
memory_pred = np.array(memory_pred)
memory_actual = np.array(memory_actual)
arith_a_pred = np.array(arith_a_pred)
arith_a_actual = np.array(arith_a_actual)
arith_b_pred = np.array(arith_b_pred)
arith_b_actual = np.array(arith_b_actual)
logical_a_pred = np.array(logical_a_pred)
logical_a_actual = np.array(logical_a_actual)
logical_b_pred = np.array(logical_b_pred)
logical_b_actual = np.array(logical_b_actual)
branching_c_pred = np.array(branching_c_pred)
branching_c_actual = np.array(branching_c_actual)
branching_u_pred = np.array(branching_u_pred)
branching_u_actual = np.array(branching_u_actual)
memory_a_pred = np.array(memory_a_pred)
memory_a_actual = np.array(memory_a_actual)
memory_b_pred = np.array(memory_b_pred)
memory_b_actual = np.array(memory_b_actual)
other_pred = np.array(other_pred)
other_actual = np.array(other_actual)
arith_node_pred = np.array(arith_node_pred)
arith_node_actual = np.array(arith_node_actual)
logical_node_pred = np.array(logical_node_pred)
logical_node_actual = np.array(logical_node_actual)
branching_node_pred = np.array(branching_node_pred)
branching_node_actual = np.array(branching_node_actual)
memory_node_pred = np.array(memory_node_pred)
memory_node_actual = np.array(memory_node_actual)
arith_node_a_pred = np.array(arith_node_a_pred)
arith_node_a_actual = np.array(arith_node_a_actual)
arith_node_b_pred = np.array(arith_node_b_pred)
arith_node_b_actual = np.array(arith_node_b_actual)
logical_node_a_pred = np.array(logical_node_a_pred)
logical_node_a_actual = np.array(logical_node_a_actual)
logical_node_b_pred = np.array(logical_node_b_pred)
logical_node_b_actual = np.array(logical_node_b_actual)
branching_node_c_pred = np.array(branching_node_c_pred)
branching_node_c_actual = np.array(branching_node_c_actual)
branching_node_u_pred = np.array(branching_node_u_pred)
branching_node_u_actual = np.array(branching_node_u_actual)
memory_node_a_pred = np.array(memory_node_a_pred)
memory_node_a_actual = np.array(memory_node_a_actual)
memory_node_b_pred = np.array(memory_node_b_pred)
memory_node_b_actual = np.array(memory_node_b_actual)
other_node_pred = np.array(other_node_pred)
other_node_actual = np.array(other_node_actual)

# Top level
print("Combined model accuracy:", metrics.accuracy_score(y_test_inst, final_pred))
print("")

# Type level
print("Arith model accuracy:", metrics.accuracy_score(arith_actual, arith_pred))
print("Logical model accuracy:", metrics.accuracy_score(logical_actual, logical_pred))
print("Branching model accuracy:", metrics.accuracy_score(branching_actual, branching_pred))
print("Memory model accuracy:", metrics.accuracy_score(memory_actual, memory_pred))
print("Other model accuracy:", metrics.accuracy_score(other_actual, other_pred))
print("")

# Inst level
print("Arith A model accuracy:", metrics.accuracy_score(arith_a_actual, arith_a_pred))
print("Arith B model accuracy:", metrics.accuracy_score(arith_b_actual, arith_b_pred))
print("Logical A model accuracy:", metrics.accuracy_score(logical_a_actual, logical_a_pred))
print("Logical B model accuracy:", metrics.accuracy_score(logical_b_actual, logical_b_pred))
print("Branching U model accuracy:", metrics.accuracy_score(branching_u_actual, branching_u_pred))
print("Branching C model accuracy:", metrics.accuracy_score(branching_c_actual, branching_c_pred))
print("Memory A model accuracy:", metrics.accuracy_score(memory_a_actual, memory_a_pred))
print("Memory B model accuracy:", metrics.accuracy_score(memory_b_actual, memory_b_pred))
print("")

# Top level
print("Class model accuracy:", metrics.accuracy_score(top_actual, top_pred))
print("")

# Type level
print("Arith node model accuracy:", metrics.accuracy_score(arith_node_actual, arith_node_pred))
print("Logical node model accuracy:", metrics.accuracy_score(logical_node_actual, logical_node_pred))
print("Branching node model accuracy:", metrics.accuracy_score(branching_node_actual, branching_node_pred))
print("Memory node model accuracy:", metrics.accuracy_score(memory_node_actual, memory_node_pred))
print("Other node model accuracy:", metrics.accuracy_score(other_node_actual, other_node_pred))
print("")

# Inst level
print("Arith A node model accuracy:", metrics.accuracy_score(arith_node_a_actual, arith_node_a_pred))
print("Arith B node model accuracy:", metrics.accuracy_score(arith_node_b_actual, arith_node_b_pred))
print("Logical A node model accuracy:", metrics.accuracy_score(logical_node_a_actual, logical_node_a_pred))
print("Logical B node model accuracy:", metrics.accuracy_score(logical_node_b_actual, logical_node_b_pred))
print("Branching U node model accuracy:", metrics.accuracy_score(branching_node_u_actual, branching_node_u_pred))
print("Branching C node model accuracy:", metrics.accuracy_score(branching_node_c_actual, branching_node_c_pred))
print("Memory A node model accuracy:", metrics.accuracy_score(memory_node_a_actual, memory_node_a_pred))
print("Memory B node model accuracy:", metrics.accuracy_score(memory_node_b_actual, memory_node_b_pred))

Combined model accuracy: 0.1515217391304348

Arith model accuracy: 0.48397836038285474
Logical model accuracy: 0.17532467532467533
Branching model accuracy: 0.39703947368421055
Memory model accuracy: 0.2125
Other model accuracy: 0.5603667855323484

Arith A model accuracy: 0.5033971587399629
Arith B model accuracy: 0.45663265306122447
Logical A model accuracy: 0.325
Logical B model accuracy: 0.32432432432432434
Branching U model accuracy: 0.19105409153952843
Branching C model accuracy: 0.22963438101347017
Memory A model accuracy: 0.2972972972972973
Memory B model accuracy: 0.42276422764227645

Class model accuracy: 0.42130434782608694

Arith node model accuracy: 0.37536412817311693
Logical node model accuracy: 0.12337662337662338
Branching node model accuracy: 0.21743421052631579
Memory node model accuracy: 0.14375
Other node model accuracy: 0.5603667855323484

Arith A node model accuracy: 0.29771463866584313
Arith B node model accuracy: 0.3278061224489796
Logical A node model accuracy:

In [24]:
# save the data and models

pickle.dump(class_model, open("class_model.sav", 'wb'))
pickle.dump(arithmetic_type_model, open("arithmetic_type_model.sav", 'wb'))
pickle.dump(arithmetic_a_model, open("arithmetic_a_model.sav", 'wb'))
pickle.dump(arithmetic_b_model, open("arithmetic_b_model.sav", 'wb'))
pickle.dump(logical_type_model, open("logical_type_model.sav", 'wb'))
pickle.dump(logical_a_model, open("logical_a_model.sav", 'wb'))
pickle.dump(logical_b_model, open("logical_b_model.sav", 'wb'))
pickle.dump(branching_type_model, open("branching_type_model.sav", 'wb'))
pickle.dump(branching_u_model, open("branching_u_model.sav", 'wb'))
pickle.dump(branching_c_model, open("branching_c_model.sav", 'wb'))
pickle.dump(memaccess_type_model, open("memaccess_type_model.sav", 'wb'))
pickle.dump(memaccess_a_model, open("memaccess_a_model.sav", 'wb'))
pickle.dump(memaccess_b_model, open("memaccess_b_model.sav", 'wb'))
pickle.dump(other_model, open("other_model.sav", 'wb'))
print("done with models")

file = open("master_data.csv", 'w')
writer = csv.writer(file)

for row in master_data_new:
    writer.writerow(row)
    
file.close()
print("done with master")


file = open("arith_data.csv", 'w')
writer = csv.writer(file)

for row in arith_data:
    writer.writerow(row)
    
file.close()
print("done with arith")

file = open("arith_a_data.csv", 'w')
writer = csv.writer(file)

for row in arith_a_data:
    writer.writerow(row)
    
file.close()
print("done with arith a")

file = open("arith_b_data.csv", 'w')
writer = csv.writer(file)

for row in arith_b_data:
    writer.writerow(row)
    
file.close()
print("done with arith b")

file = open("logical_data.csv", 'w')
writer = csv.writer(file)

for row in logical_data:
    writer.writerow(row)
    
file.close()
print("done with log")

file = open("logical_a_data.csv", 'w')
writer = csv.writer(file)

for row in logical_a_data:
    writer.writerow(row)
    
file.close()
print("done with log a")

file = open("logical_b_data.csv", 'w')
writer = csv.writer(file)

for row in logical_b_data:
    writer.writerow(row)
    
file.close()
print("done with log b")

file = open("branching_data.csv", 'w')
writer = csv.writer(file)

for row in branching_data:
    writer.writerow(row)
    
file.close()
print("done with branch")

file = open("branching_c_data.csv", 'w')
writer = csv.writer(file)

for row in branching_c_data:
    writer.writerow(row)
    
file.close()
print("done with branch c")

file = open("branching_u_data.csv", 'w')
writer = csv.writer(file)

for row in branching_u_data:
    writer.writerow(row)
    
file.close()
print("done with branch u")

file = open("memory_data.csv", 'w')
writer = csv.writer(file)

for row in memory_data:
    writer.writerow(row)
    
file.close()
print("done with mem")

file = open("memory_a_data.csv", 'w')
writer = csv.writer(file)

for row in memory_a_data:
    writer.writerow(row)
    
file.close()
print("done with mem a")

file = open("memory_b_data.csv", 'w')
writer = csv.writer(file)

for row in memory_b_data:
    writer.writerow(row)
    
file.close()
print("done with mem b")

file = open("other_data.csv", 'w')
writer = csv.writer(file)

for row in other_data:
    writer.writerow(row)
    
file.close()
print("done with other")

done with models
done with master
done with arith
done with arith a
done with arith b
done with log
done with log a
done with log b
done with branch
done with branch c
done with branch u
done with mem
done with mem a
done with mem b
done with other
